# All imports

In [ ]:
import cv2
import os
import random
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Concatenate, Input, Add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Drive Set up

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_dirs = {
    'rotten': '/content/drive/MyDrive/data/rotten',
    'mid': '/content/drive/MyDrive/data/mid',
    'fresh': '/content/drive/MyDrive/data/fresh'
}

In [ ]:
output_dirs = {
    'rotten': '/content/drive/MyDrive/data/rotten_frames',
    'mid': '/content/drive/MyDrive/data/mid_frames',
    'fresh': '/content/drive/MyDrive/data/fresh_frames'
}

In [ ]:
for category, out_dir in output_dirs.items():
    os.makedirs(out_dir, exist_ok=True)

# Dataset Preparation

## Frame Extraction

In [ ]:
fresh_frames_dir = '/content/drive/MyDrive/data/fresh_frames'
mid_frames_dir = '/content/drive/MyDrive/data/mid_frames'
rotten_frames_dir = '/content/drive/MyDrive/data/rotten_frames'

fresh_images_count = len([file for file in os.listdir(fresh_frames_dir) if file.endswith('.jpg')])
mid_images_count = len([file for file in os.listdir(mid_frames_dir) if file.endswith('.jpg')])
rotten_images_count = len([file for file in os.listdir(rotten_frames_dir) if file.endswith('.jpg')])

print(f"Number of images in fresh_frames: {fresh_images_count}")
print(f"Number of images in mid_frames: {mid_images_count}")
print(f"Number of images in rotten_frames: {rotten_images_count}")

Number of images in fresh_frames: 110
Number of images in mid_frames: 130
Number of images in rotten_frames: 60


## Data preprocessing

In [ ]:
base_path = "/content/drive/MyDrive/data/augmented"

folders = {
    "fresh": "augmented_fresh_frames",
    "mid": "augmented_mid_frames",
    "rotten": "augmented_rotten_frames"
}

for label, folder in folders.items():
    folder_path = os.path.join(base_path, folder)
    count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    print(f"Number of images in {folder}: {count}")

Number of images in augmented_fresh_frames: 1100
Number of images in augmented_mid_frames: 1300
Number of images in augmented_rotten_frames: 600


In [ ]:
rotten_dir = os.path.join(base_path, 'augmented_rotten_frames')
mid_dir = os.path.join(base_path, 'augmented_mid_frames')
fresh_dir = os.path.join(base_path, 'augmented_fresh_frames')

In [ ]:
# Class weights (handle imbalance)
total_images = {
    'rotten': 600,
    'mid': 1300,
    'fresh': 1100
}

total = sum(total_images.values())

# 0 - rotten, 1 - mid, 2 - fresh
class_weights = {
    0: int(total / total_images['rotten']),  # rotten
    1: int(total / total_images['mid']),     # mid
    2: int(total / total_images['fresh'])    # fresh
}

print("Class Weights:", class_weights)

Class Weights: {0: 5, 1: 2, 2: 2}


In [ ]:
batch_size = 32
img_size = (224, 224)

datagen = ImageDataGenerator(
    validation_split=0.3,
)
#Training data
train_data = datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
#Validation data
val_data = datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2100 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


# Models Training

## VGG16

In [ ]:
#RGB to BGR
from tensorflow.keras.applications.vgg16 import preprocess_input

datagen_vgg = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.3
)

#Loading training data
train_data_vgg = datagen_vgg.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

#Loading validation data
val_data_vgg = datagen_vgg.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2100 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import VGG16

#Loading vgg16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the base layers
for layer in base_model.layers:
    layer.trainable = False

#Model Building
model_vgg = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Compilation
model_vgg.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#Model summary
model_vgg.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,846,787 (56.64 MB)

 Trainable params: 132,099 (516.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import time

# Callbacks
checkpoint_vgg = ModelCheckpoint(
    "vgg_cpu.h5",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr_vgg = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

early_stopping_vgg = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Start training
start_vgg = time.time()

history_vgg = model_vgg.fit(
    train_data_vgg,
    epochs=50,
    validation_data=val_data_vgg,
    callbacks=[checkpoint_vgg, reduce_lr_vgg, early_stopping_vgg]
)

end_vgg = time.time()
runtime_vgg = end_vgg - start_vgg

print(f"Training Time for VGG16 model: {runtime_vgg:.2f} seconds")

Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.4607 - loss: 3.3713 
Epoch 1: val_accuracy improved from -inf to 0.87667, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 2040s 31s/step - accuracy: 0.4624 - loss: 3.3545 - val_accuracy: 0.8767 - val_loss: 0.4108 - learning_rate: 1.0000e-04
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.7752 - loss: 1.0767 
Epoch 2: val_accuracy improved from 0.87667 to 0.90444, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1973s 30s/step - accuracy: 0.7754 - loss: 1.0747 - val_accuracy: 0.9044 - val_loss: 0.3522 - learning_rate: 1.0000e-04
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.8245 - loss: 0.7235 
Epoch 3: val_accuracy improved from 0.90444 to 0.91889, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1897s 29s/step - accuracy: 0.8247 - loss: 0.7234 - val_accuracy: 0.9189 - val_loss: 0.2602 - learning_rate: 1.0000e-04
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.8623 - loss: 0.5721 
Epoch 4: val_accuracy improved from 0.91889 to 0.92667, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1901s 29s/step - accuracy: 0.8623 - loss: 0.5716 - val_accuracy: 0.9267 - val_loss: 0.1936 - learning_rate: 1.0000e-04
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.8695 - loss: 0.5074 
Epoch 5: val_accuracy improved from 0.92667 to 0.92778, saving model to vgg_cpu.h5


...

66/66 ━━━━━━━━━━━━━━━━━━━━ 1926s 29s/step - accuracy: 0.9253 - loss: 0.2059 - val_accuracy: 0.9467 - val_loss: 0.1196 - learning_rate: 2.0000e-05
Epoch 13/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.9292 - loss: 0.2234 
Epoch 13: val_accuracy improved from 0.94667 to 0.94778, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1948s 30s/step - accuracy: 0.9293 - loss: 0.2232 - val_accuracy: 0.9478 - val_loss: 0.1159 - learning_rate: 2.0000e-05
Epoch 14/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.9270 - loss: 0.2234 
Epoch 14: val_accuracy improved from 0.94778 to 0.94889, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1908s 29s/step - accuracy: 0.9270 - loss: 0.2232 - val_accuracy: 0.9489 - val_loss: 0.1145 - learning_rate: 2.0000e-05
Epoch 15/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.9298 - loss: 0.2131 
Epoch 15: val_accuracy improved from 0.94889 to 0.95111, saving model to vgg_cpu.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 1936s 29s/step - accuracy: 0.9298 - loss: 0.2129 - val_accuracy: 0.9511 - val_loss: 0.1139 - learning_rate: 2.0000e-05
Epoch 16/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.9390 - loss: 0.1583 
Epoch 16: val_accuracy did not improve from 0.95111
66/66 ━━━━━━━━━━━━━━━━━━━━ 1936s 29s/step - accuracy: 0.9390 - loss: 0.1586 - val_accuracy: 0.9478 - val_loss: 0.1125 - learning_rate: 2.0000e-05
Epoch 17/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.9228 - loss: 0.1848 
Epoch 17: val_accuracy did not improve from 0.95111
66/66 ━━━━━━━━━━━━━━━━━━━━ 1920s 29s/step - accuracy: 0.9229 - loss: 0.1849 - val_accuracy: 0.9500 - val_loss: 0.1151 - learning_rate: 2.0000e-05
Epoch 18/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.9442 - loss: 0.1502 
Epoch 18: val_accuracy did not improve from 0.95111

Epoch 18: ReduceLROnPlateau reducing learning rate to 3.999999898951501e-06.
66/66 ━━━━━━━━━━━━━━━━━━━━ 1960s 30s/step - accuracy: 0.9440 - loss: 0

66/66 ━━━━━━━━━━━━━━━━━━━━ 1904s 29s/step - accuracy: 0.9322 - loss: 0.1996 - val_accuracy: 0.9533 - val_loss: 0.1130 - learning_rate: 4.0000e-06
Epoch 22/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.9356 - loss: 0.1772 
Epoch 22: val_accuracy did not improve from 0.95333
66/66 ━━━━━━━━━━━━━━━━━━━━ 1963s 30s/step - accuracy: 0.9357 - loss: 0.1771 - val_accuracy: 0.9533 - val_loss: 0.1126 - learning_rate: 4.0000e-06
Epoch 23/50
 4/66 ━━━━━━━━━━━━━━━━━━━━ 20:52 20s/step - accuracy: 0.9284 - loss: 0.2011

In [ ]:
# model_vgg = load_model("vgg_cpu.h5")